## 3.3. Synthetic Regression Data 线性回归的简洁实现

在过去的几年里，出于对深度学习强烈的兴趣， 许多公司、学者和业余爱好者开发了各种成熟的开源框架。 这些框架可以自动化基于梯度的学习算法中重复性的工作。 在 3.2节中，我们只运用了：
  （1）通过张量来进行数据存储和线性代数；
  （2）通过自动微分来计算梯度。 
实际上，由于数据迭代器、损失函数、优化器和神经网络层很常用， 现代深度学习库也为我们实现了这些组件。

本节将介绍如何通过使用深度学习框架来简洁地实现 3.2节中的线性回归模型。

### 3.3.1. Generating the Dataset 生成数据集

In [1]:
from mxnet import autograd, gluon, np, npx
import tools_func as d2l

npx.set_np()

true_w = np.array([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

[10:32:03] ../src/storage/storage.cc:202: Using Pooled (Naive) StorageManager for CPU


### 3.3.2. 读取数据集

我们可以调用框架中现有的API来读取数据。 我们将features和labels作为API的参数传递，并通过数据迭代器指定batch_size。 此外，布尔值is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据。

In [2]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个Gluon数据迭代器"""
    dataset = gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

# 读取并打印第一个小批量样本
next(iter(data_iter))

[array([[-1.4747815 , -0.22913499],
        [ 0.08104636, -0.28282833],
        [-1.6765159 , -0.35567233],
        [ 0.4179527 ,  0.25945112],
        [-1.4976466 , -0.8314139 ],
        [ 0.8753208 ,  0.30068344],
        [ 0.39020136, -1.9421831 ],
        [ 0.6174898 , -1.6827948 ],
        [ 1.8816522 ,  0.3765186 ],
        [ 1.6839302 ,  0.7737382 ]]),
 array([[ 2.042786 ],
        [ 5.3282986],
        [ 2.0601528],
        [ 4.129761 ],
        [ 4.029345 ],
        [ 4.924385 ],
        [11.577917 ],
        [11.155544 ],
        [ 6.6707735],
        [ 4.9390917]])]

### 3.3.3. Defining the Model 定义模型

当我们在 3.2节中实现线性回归时， 我们明确定义了模型参数变量，并编写了计算的代码，这样通过基本的线性代数运算得到输出。 但是，如果模型变得更加复杂，且当我们几乎每天都需要实现模型时，自然会想简化这个过程。 这种情况类似于为自己的博客从零开始编写网页。 做一两次是有益的，但如果每个新博客就需要工程师花一个月的时间重新开始编写网页，那并不高效。

对于标准深度学习模型，我们可以使用框架的预定义好的层。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。 我们首先定义一个模型变量net，它是一个Sequential类的实例。 Sequential类将多个层串联在一起。 当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推。 在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential。 但是由于以后几乎所有的模型都是多层的，在这里使用Sequential会让你熟悉“标准的流水线”。

回顾 图3.1.2中的单层网络架构， 这一单层被称为全连接层（fully-connected layer）， 因为它的每一个输入都通过矩阵-向量乘法得到它的每个输出。

